In [ ]:
#Objective: Find BlackList of defconf,board-name

In [ ]:
import pandas as pd
import json
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#Input Parameters
job = 'next'
date_range = 15
offence_threshold = 5

In [ ]:
import requests
from urlparse import urljoin
import kernelci_api_key

BACKEND_URL = "http://api.kernelci.org"

def invoke():
    headers = {"Authorization":kernelci_api_key.getkernelcikey()}
    params = {
            "job": job,
            "date_range": date_range
        }
    url = urljoin(BACKEND_URL, "/boot")
    response = requests.get(url, params=params, headers=headers)
    return response.content

In [ ]:
#Plot above with interaction
from bokeh.plotting import figure, output_file, output_notebook,show
from bokeh.models import ColumnDataSource, Circle, HoverTool,CustomJS

def draw_interactive(df, defconfig, board):
    (x,y,k) = (df.index.to_series(), df['status_fl'],df['git_describe'])

    #have to do this as Boken can't print Dates properly on Hover
    ts = pd.to_datetime(x.values)
    z = ts.strftime('%Y.%m.%d')

    output_notebook()
    p = figure(width=600,height=300,x_axis_type='datetime',
               title=defconfig+'___'+board)
    p.line(x,y,line_dash="4 4", line_width=2,color='gray')

    source = ColumnDataSource({'x':x,'y':y,'z':z,'k':k})

    invisible_circle = Circle(x='x',y='y', fill_color='gray', fill_alpha=0.1, line_color=None, size=10)
    visible_circle = Circle(x='x',y='y', fill_color='firebrick', fill_alpha=0.5, line_color=None, size=10)

    cr = p.add_glyph(source, invisible_circle, selection_glyph=visible_circle, nonselection_glyph=invisible_circle)

    #Add hover tool, that selects the circle
    # Add a hover tool, that selects the circle
    code = "source.set('selected', cb_data['index']);"

    callback = CustomJS(args={'source': source}, code=code)

    p.add_tools(HoverTool(
                tooltips=[
                            ("d", "@z"),
                            ("stat", "@y"),
                            ('gd', "@k")
                        ], 
                callback=callback, 
                renderers=[cr], 
                mode='vline'))
    show(p)
           

In [ ]:
content = invoke()
contentjs = json.loads(content)
df = json_normalize(contentjs['result'])

In [ ]:
#Select only columns we care about for this work
df2 = df[['_id.$oid',u'arch',u'board',u'board_instance',
           u'created_on.$date',u'defconfig',u'dtb',
           u'git_describe',u'lab_name',u'mach',u'status']]

#convert created_on to datetime and use as index
df2['created_on'] = pd.to_datetime(df2['created_on.$date'],unit='ms')
df2 = df2.set_index('created_on')
df2 = df2.drop('created_on.$date',axis=1)

#Consider only Pass and Fail
df2 = df2[df2.status.isin(['PASS','FAIL'])]
df2['status_fl'] = df2.status.map({'PASS':1,'FAIL':0})

del df
df2.head()

In [ ]:
df3 = df2.groupby(['defconfig','board']).mean()

In [ ]:
df3['offence_score'] = (1 - df3.status_fl)*100
df5 = df3[df3.offence_score > offence_threshold]
df5

In [ ]:
#Understand why certain combination is really offensive
defconfig = 'exynos_defconfig'
board = 'exynos5250-arndale'

df4 = df2[(df2.defconfig == defconfig) & (df2.board == board)].sort_index()
draw_interactive(df4,defconfig,board)

In [ ]:
for x in df5.reset_index().values:
    df4 = df2[(df2.defconfig == x[0]) & (df2.board == x[1])].sort_index()
    draw_interactive(df4,x[0],x[1])